# ECON457 Lab03

## Solve a system of linear equations:  
- In Scipy 
- In Sympy

In [4]:
# Usual preliminary calls
import numpy as np
import scipy as sp
import scipy.linalg as la 
# The top-level components of Scipy (such as linalg, optimize, etc.) are so-called subpackages and not modules 
# (i.e., they're directories, not source code). 
# Scipy is set up such that subpackages must be imported separately: for example, sp.linalg won't work

import sympy as sym
sym.init_printing() # for printing latex/mathjax format output

## 1. One key difference between numerical computation and symbolic computation

### 1.1 Numerical Computation
Exact arithmetic and computer arithmetic do not always give the same answers (even in programs without programming errors).

In [5]:
1/3+1/2 == 5/6

False

In [6]:
1/3+1/2

In [7]:
5/6

### 1.2 Symbolic Computation
Sympy provides symbolic computation.

In [8]:
sym.Rational(1,3) + sym.Rational(1,2)

In [11]:
sym.Rational(1,3) + sym.Rational(1,2) == sym.Rational(5,6) 

True

## 2. Systems of linear equations

 ###  2.1 Express a system of linear equations in matrix form
 
 $$A x = b$$

np.matrix() 
returns a matrix from an array-like object, or from a string of data.

A [matrix](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.matrix.html) is a specialized 2-D array that retains its 2-D nature through operations. It has certain special operators, such as *matrix multiplication* and *matrix power*. It also has some **fields/attributes**: `shape, ndim, size`.

###### On a side note: Numpy [array](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.creation.html) is more versatile and can be of high dimension.


Let's consider some common linear algebra commands, and the related output:

In [59]:
A = np.matrix([[1, 0.0005], [1, -0.00025]])
A

matrix([[ 1.0e+00,  5.0e-04],
        [ 1.0e+00, -2.5e-04]])

In [60]:
A.shape

In [14]:
A.ndim

In [15]:
A.dtype

dtype('float64')

In [16]:
A.size

In [17]:
b = np.array([0.55, -0.2])
b

array([ 0.55, -0.2 ])

In [18]:
b.shape

In [19]:
b.ndim

In [20]:
b = np.matrix([0.55, -0.2])
b

matrix([[ 0.55, -0.2 ]])

In [21]:
b.shape

In [22]:
b = np.matrix([[0.55], [-0.2]])
b

matrix([[ 0.55],
        [-0.2 ]])

In [23]:
b.shape

In [24]:
b.ndim

### 2.2 Solve the system by using Scipy

Scipy has a set of tools to solve a system of linear equations in its `linalg` subpackages. 

$$A x =b$$


#### 2.2.1 Method 1
Compute the inverse of the matrix **A** and solve **x** via $A^{-1}b$.

In [25]:
#Compute and display the inverse 
A_inv = la.inv(A) 
A_inv

array([[ 3.33333333e-01,  6.66666667e-01],
       [ 1.33333333e+03, -1.33333333e+03]])

In [26]:
x = np.dot(A_inv, b)
x

matrix([[5.e-02],
        [1.e+03]])

In [27]:
A_inv@b # the symbol @ works as np.dot in python3.5+ 

matrix([[5.e-02],
        [1.e+03]])

#### 2.2.2 Method 2
We can solve for **x** directly by using the solver

In [28]:
x = la.solve(A, b)
x   

array([[5.e-02],
       [1.e+03]])

#### 2.2.3  Method 3

Solve for **x** via the LU decomposition:

```python
"""
Solve a system of linear equations by the LU-decomposition
Comes from LU decomposition of a matrix A s.t. A = LU
Then
LUx = b => Ux = y => Ly = b
"""
p, l, u = la.lu(A)
```

The following example is from the slides used in the lecture.

##### 2.2.3.1 PLU decomposition of the matrix A

In [29]:
A = np.matrix([[-3, 2, 3], 
               [-3 ,2 ,1], 
               [3, 0 ,0]])
A

matrix([[-3,  2,  3],
        [-3,  2,  1],
        [ 3,  0,  0]])

In [30]:
b = np.matrix([[10], [8], [-3]])
b

matrix([[10],
        [ 8],
        [-3]])

In [38]:
p,l, u = la.lu(A)

In [39]:
p

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [40]:
l

array([[ 1.,  0.,  0.],
       [-1.,  1.,  0.],
       [ 1.,  0.,  1.]])

In [41]:
u

array([[-3.,  2.,  3.],
       [ 0.,  2.,  3.],
       [ 0.,  0., -2.]])

##### 2.2.3.2 Check that the decomposition works

In [42]:
# the symbol @ works as np.dot in python3.5+
p@l@u 

array([[-3.,  2.,  3.],
       [-3.,  2.,  1.],
       [ 3.,  0.,  0.]])

In [43]:
# Let's use the .dot syntax instead
print(np.dot(p, np.dot(l, u)))
p.dot(l.dot(u))

[[-3.  2.  3.]
 [-3.  2.  1.]
 [ 3.  0.  0.]]


array([[-3.,  2.,  3.],
       [-3.,  2.,  1.],
       [ 3.,  0.,  0.]])

In [44]:
#Note: the symbol * performs a different matrix multiplication (element by element)
p*l*u

array([[-3.,  0.,  0.],
       [-0.,  0.,  0.],
       [ 0.,  0., -0.]])

##### 2.2.3.3 Using the lower triangular matrix to solve for y (forward substitution)

$$ply = b$$

In [45]:
y = la.solve(p@l, b)
y   

array([[10.],
       [ 7.],
       [-2.]])

##### 2.2.3.4 Using the upper triangular matrix and y to solve for x (backward substitution)
$$ux= y$$

In [46]:
x = la.solve(u, y)
x   

array([[-1.],
       [ 2.],
       [ 1.]])

### 2.3 Solve the system with Sympy

#### 2.3.1 Preliminaries

In [47]:
# Rename A and b as M and v
M = sym.Matrix(A)
v = sym.Matrix(b)
M

⎡-3  2  3⎤
⎢        ⎥
⎢-3  2  1⎥
⎢        ⎥
⎣3   0  0⎦

In [48]:
v

⎡10⎤
⎢  ⎥
⎢8 ⎥
⎢  ⎥
⎣-3⎦

#### 2.3.2 Method 1

Compute the inverse of the matrix **A** and solve for **x** via $A^{-1}b$.

In [49]:
# In SymPy the inverse is computed by Gaussian elimination by default
M.inv()

⎡ 0     0    1/3⎤
⎢               ⎥
⎢-1/4  3/4   1/2⎥
⎢               ⎥
⎣1/2   -1/2   0 ⎦

In [50]:
#Now the selected option "LU" allows to use the LU decomposition, which is more efficient 
#Note: we get the same result
M.inv("LU") 

⎡ 0     0    1/3⎤
⎢               ⎥
⎢-1/4  3/4   1/2⎥
⎢               ⎥
⎣1/2   -1/2   0 ⎦

In [51]:
#Now we can compute the vector that solves the system
M.inv("LU")*v

⎡-1⎤
⎢  ⎥
⎢2 ⎥
⎢  ⎥
⎣1 ⎦

#### 2.3.3 Method 2  
Similar to SciPy, we can solve for **x** directly by using the LU solver, called LUsolve

In [52]:
x=M.LUsolve(v)
x

⎡-1⎤
⎢  ⎥
⎢2 ⎥
⎢  ⎥
⎣1 ⎦

#### 2.3.4 Method 3

Solve for **x** via the LU decomposition.

##### 2.3.4.1 Obtain L, U, and the row permutation matrix

In [53]:
M.LUdecomposition()

⎛⎡1   0  0⎤  ⎡-3  2  3 ⎤          ⎞
⎜⎢        ⎥  ⎢         ⎥          ⎟
⎜⎢-1  1  0⎥, ⎢0   2  3 ⎥, [[1, 2]]⎟
⎜⎢        ⎥  ⎢         ⎥          ⎟
⎝⎣1   0  1⎦  ⎣0   0  -2⎦          ⎠


Returns (L, U, perm) where L is a lower triangular matrix with a unit main diagonal, U is an upper triangular matrix, and perm is a list of row swap index pairs. If A is the original matrix, then A = (L*U).permuteBkwd(perm), and the row permutation matrix P such that P*A = L*U can be computed by P=eye(A.row).permuteFwd(perm).

$$P*A = L*U$$

See the documentation for LUCombined for details about the keyword argument rankcheck, iszerofunc, and simpfunc.

http://docs.sympy.org/latest/modules/matrices/matrices.html

In [54]:
L,U,perm = M.LUdecomposition()
L

⎡1   0  0⎤
⎢        ⎥
⎢-1  1  0⎥
⎢        ⎥
⎣1   0  1⎦

In [55]:
U

⎡-3  2  3 ⎤
⎢         ⎥
⎢0   2  3 ⎥
⎢         ⎥
⎣0   0  -2⎦

##### 2.3.4.2  Let's adjust the RHS using the permutation matrix

In [56]:
v = v.permuteBkwd(perm)
v

⎡10⎤
⎢  ⎥
⎢-3⎥
⎢  ⎥
⎣8 ⎦

##### 2.3.4.3 To obtain the elements of x, first use the lower triangular matrix to solve for y (forward substitution)

In [57]:
L.LUsolve(v)

⎡10⎤
⎢  ⎥
⎢7 ⎥
⎢  ⎥
⎣-2⎦

##### 2.3.4.4 Then continue to use the upper triangular matrix and y to solve for x (backward substitution))

In [58]:
U.LUsolve(L.LUsolve(v))

⎡-1⎤
⎢  ⎥
⎢2 ⎥
⎢  ⎥
⎣1 ⎦